# Regression using Cyclic Boosting

First, install the  package and its dependencies

```sh
!pip install cyclic-boosting
```

In [1]:
import pandas as pd
import numpy as np
import os

Let's use the test dataset from kaggle

For time-series data, a "date" column must be included to indicate the date and time the data was obtained. The column name and format must be consistent. The "dayofweek" column for the day of the week and the "dayofyear" column for the total number of days in the year are automatically created if not already present, but if they are already present, the column names must be correct.

This dataset has daily data. In this dataset, the "instant" column is the data number. The "casual" and "registered" columns are the breakdown of sales, so they should be deleted.

In [2]:
parpath = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
df = pd.read_csv(parpath + "/bike_sharing_data/day.csv")
df = df.rename(columns={'dteday': 'date', 'weekday': 'dayofweek'})
df = df.drop(columns=['instant', 'casual', 'registered'])
df["date"] = pd.to_datetime(df["date"])

df.to_csv("./bike_sharing_day.csv", index=False)

In [3]:
df.head()

,date,season,yr,mnth,holiday,dayofweek,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600


# Automated Machine Learning with Tornado
With tornado, you can automatically perform data preparation, feature property setting, hyperparameter tuning, model building, training, evaluation, and plotting!

In [4]:
from cyclic_boosting.tornado import Generator, Manager, Trainer

data_deliverler = Generator.TornadoDataModule("./bike_sharing_day.csv")
manager = Manager.TornadoVariableSelectionModule()
trainer = Trainer.SqueezeTrainer(data_deliverler, manager)
trainer.run(target="cnt", log_policy="compute_COD", verbose=False)

Data interval is 'daily'. If not, give
    the data_interval option in the TornadoDataModule.

Auto analysis target ['temp', 'atemp', 'hum', 'windspeed']
    has_trend: ['temp', 'atemp', 'windspeed']
    has_seasonality: []
    has_up_monotonicity: []
    has_down_monotonicity: []
    has_linearity: []
    has_missing: []


iter: 54 / 77 Encountered negative change of loss. This might not be a problem, as long as the model converges. Check the LOSS changes in the analysis plots.
iter: 77 / 77 
TRUNCATED
['atemp', 'temp', 'mnth', 'season', 'yr', 'weathersit', 'hum', 'windspeed', 'holiday', 'workingday', 'dayofweek', 'dayofyear', ('yr', 'atemp'), ('yr', 'temp'), ('yr', 'mnth'), ('season', 'yr'), ('weathersit', 'atemp'), ('weathersit', 'temp'), ('season', 'weathersit'), ('mnth', 'weathersit'), ('workingday', 'atemp'), ('workingday', 'temp'), ('season', 'atemp'), ('holiday', 'atemp'), ('yr', 'weathersit'), ('season', 'temp'), ('season', 'hum'), ('dayofweek', 'temp'), ('season', 'workingday'), ('mnth', 'holiday'), ('mnth', 'workingday'), ('temp', 'atemp'), ('mnth', 'atemp'), ('mnth', 'temp'), ('season', 'holiday'), ('yr', 'windspeed'), ('yr', 'workingday'), ('yr', 'holiday'), ('season', 'dayofweek'), ('season', 'mnth'), ('yr', 'dayofweek'), ('season', 'windspeed'), ('mnth', 'dayofweek'), ('holiday', 'weathersi

# Load the best model and make predictions.

Get the best model path.

In [5]:
import pickle
from pathlib import Path

model_nos = []
for p in sorted(Path("./models/").glob("model*")):
    model_nos.append(str(p)[str(p).find("_") + 1 :])
model_path = f"./models/model_{model_nos[-1]}/model_{model_nos[-1]}.pkl"
print(model_path)

./models/model_58/model_58.pkl


Make predictions with the best model.

In [6]:
data = {
    'season': [4],
    'yr': [0],
    'mnth': [11],
    'holiday': [0],
    'workingday': [1],
    'weathersit': [2],
    'temp': [0.341667],
    'atemp': [0.323221],
    'hum': [0.575833],
    'windspeed': [0.305362],
    'dayofweek': [4],
    'dayofyear': [180],
}
X = pd.DataFrame(data)

with open(model_path, "rb") as f:
    CB_est = pickle.load(f)
    yhat = CB_est.predict(X.copy())
    print(yhat)

[3072.31800301]
